# Jupyter Notebook GenoRobotics Full Pipeline

## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
import os.path as ospath
from lib.consensus.consensus import run_consensus
from lib.identification.identification import run_identification
from lib.general_helpers.process_fastq import concatenate_fastq
from lib.demultiplexing.run_demultiplexing import run_demultiplexing
import os.path as ospath
from Bio import SeqIO

/home/ghassan_unix/anaconda3/envs/genorobotics_pipeline/lib/python3.10/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


## Define Your File and Folder Paths

Modify `windows` if you're on Windows and want to use WSL from windows (you are not running this code from wsl).
Specify whether you want to run the demultiplexing step.

If the fastq file you want to choose is part of an expedition, you cna easily choose it by filling in the input expedition folder name and barcode

In [2]:
#for choosing a fastq that is part of an expedition easily
input_expedition_folder = "summer_expedition"
barcode_nb = 5
windows = False
demultiplexing=True
genes =["matK","psbA-trnH","rbcL"]

In [3]:
input_expedition_path= ospath.join("data", input_expedition_folder)

for root, dirs, files in os.walk(input_expedition_path):
    if root.endswith(str(f"barcode{barcode_nb}")):
        input_folder_path = root

for _,_, files in os.walk(input_folder_path):
    for file in files:
        if file.endswith(".fastq"):
            input_fastq_path = ospath.join(input_folder_path, file)
            input_fastq_filename=file
        if file.endswith(".fasta"):
            input_ref_path = ospath.join(input_folder_path, file)
        

print(f"pipeline will run on file {input_fastq_path}")
base_name = input_fastq_filename[:-6]

output_path = ospath.join("output",base_name)
print(f"results can be found in {output_path}")

pipeline will run on file data/summer_expedition/Berberis_aquifolium_matK_rbcL_trnH-psbA_ITS_barcode5/Berberis_aquifoliummatK_rbcL_trnH-psbA_ITS_barcode5.fastq
results can be found in output/Berberis_aquifoliummatK_rbcL_trnH-psbA_ITS_barcode5


If the fastq file you want to choose is not part of an expedition, you can directly fill in its name. **You should run either the previous two cells if it's part of an expedition, or the following cell if it's not, but never the three of them**

In [6]:
#for standalone fastq files outside any expedition
windows = False
demultiplexing=True
input_fastq_file = "Convallaria_magalistrnH-psbA_barcode88.fastq"
input_expedition_folder = None

input_fastq_path = ospath.join("data", input_fastq_file)
base_name = ospath.splitext(input_fastq_file)[0]
output_path = ospath.join("output",base_name)

## Quality Control

As a control, we check if the fastq file selected contains enough reads, aka. contains more reads than a certain threshold

In [4]:
threshold_fastq = 10

too_small = True
n=0
for read in SeqIO.parse(input_fastq_path, "fastq"):
    n+=1
    if n >= threshold_fastq:
        too_small = False
        break

if too_small:
    raise ValueError(f"The fastq file you selected contains {n} reads, less than the set threshold of {threshold_fastq} to run pipeline")

print("Passed control!")


Passed control!


## Run Demultiplexing

In [7]:
demultiplexing_path = ospath.join(output_path,"demultiplexing")
run_demultiplexing(base_name,input_fastq_path,demultiplexing_path,genes=genes)

Logging set up at output/Berberis_aquifoliummatK_rbcL_trnH-psbA_ITS_barcode5/demultiplexing/Berberis_aquifoliummatK_rbcL_trnH-psbA_ITS_barcode5_demultiplexing_pipeline_log.log


ValueError: Your XML file did not start with '<?xml'... but instead '2024-04-27 09:18:10,'

## Run Consensus Sequence Generation

Select which consensus sequence generation method you want to use by setting the "consensus_method" variable to either:

- "majority" (default)

- "consensus"

- "consensus_with_ambiguities"

In [ ]:
# choose a consensus method between the following:
# - "80_20_best_sequence"
# - "80_20_longest_sequence"
# - "straightforward_best_sequence"

# consensus_method = "straightforward_best_sequence"
consensus_method = "80_20_best_sequence"
consensus_output_path= ospath.join(output_path,"consensus")
file_names= os.listdir(demultiplexing_path)
for file in file_names:
    if file.endswith(".fastq"):
        gene_name = ospath.basename(file)[:-6]
        demultiplexed_fastq_path=ospath.join(demultiplexing_path,file)
        demultiplexed_consensus_fastq_folder_path=ospath.join(consensus_output_path,base_name)
        run_consensus(gene_name,demultiplexed_fastq_path,consensus_method,None,demultiplexed_consensus_fastq_folder_path)
   


## Run Identification of Consensus Sequence
- Run the following cell to identify the consensus sequence.
- Change db to the database you want to use. Options are "matK", "rbcL", "psbA-trnH" and "ITS". If you want to use all of them, set db to None.

In [ ]:
# choose an identification method between the following:
# - "blastn"

identification_method = "blastn"
        final_consensus_path= os.path.join(demultiplexed_consensus_fastq_folder_path, f"{gene_name}_final_consensus.fasta")
        run_identification(base_name,None,)
for 
# Choose your db along the genes you're trying to identify : matK, rbcL, psbA-trnH, ITS
db = set(genes)

run_identification(base_name,expedition_name=input_expedition_folder, db=db, identification_method=identification_method, windows=windows)

print("Pipeline finished !")
print(f"results can be found in {output_path}")